<a href="https://colab.research.google.com/github/evansemet/IBM-Ponder-This-Solutions/blob/main/2023_06_The_Temporal_Cheese_Maze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A super intelligent mouse is placed in a three dimensional "maze", consisting of a $k\times k\times k$ grid, where the mouse is placed at the beginning at (1,1,1) at time $t=1$.

Every cell in the maze contains one unit of cheese. The catch: This is **temporal** cheese; it's phasing in and out, and is present only at certain times. If the mouse is at cell (a,b,c) at time t where the cheese is present in the cell, the mouse collects it. Afterwards, the cheese **no longer appears** in that cell!

At each time unit, the mouse can either stay in place, or move one step in exactly one direction. The maze has walls allowing only movement right, up, and forward on the X, Y, and Z axes, respectively. For example, if present at (3,5,1) at $t=13$, the possible locations for the mouse at $t=14$ are

1. (3,5,1) (\*\*waiting\*\* in place)

2. (4,5,1) (moving \*\*right\*\* in the X axis)

3. (3,6,1) (moving \*\*up\*\* in the Y axis)

4. (3,5,2) (moving \*\*forward\*\* in the Z axis)

We denote the path taken by the mouse by a sequence of letters: 'W', 'R', 'U', and 'F', corresponding to options 1-4 above.

At time n, the mouse is removed from the maze and can no longer collect cheese. His goal: To maximize the cheese collected by then.

The phases of the cheese are determined by the following linear congruential generator:

$f(x) = (ax+c)\ mod\ m$

Where

$a = 1103515245$

$c = 12345$

$m = 2^{31}$

The cheese is present at (a,b,c) at time t if and only if there is a value $0\le x < n/2$ such that $t=f(a\cdot b\cdot c + x)\ mod\ n $.

For example, when $n=20$ and (a,b,c) = (3,5,1) , the set of values of t where the cheese appears is $\{1, 3, 4, 5, 6, 7, 8, 9, 10, 12\}$.

For $k=5$ and $n=20$, the maximum amount of cheese obtainable is 10 units, given by the following path:

FFRFWFWRRRUUUWWWUWW

**Your goal**: Find the maximum amount of cheese obtainable for $k=30$ and $n=100$. Supply your solution as two lines, one with the maximum number of cheese units and the other with the path itself.

**A bonus** "*" will be given for solving the same problem for $k=50$ and $n=200$, but this time ***allowing** the mouse to move in more possible ways: **left** (L), **down** (D) and **backwards** (B) as well as **wait** (W), **right** (R), **up** (U) and **forward** (F), and **allowing** the mouse to collect the cheese more than once for the same cell, at each time where the cheese is present there.

In [ ]:
# install necessary packages
# pip install is needed for bitarray in Collab
!pip install bitarray
from bitarray import bitarray
import numpy as np
from itertools import product
from time import time

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Given values
A = 1103515245
C = 12345
M = 1 << 31

k = 30
n = 100

In [ ]:
'''
calculates the times when cheese is available at a given coordinate
@param a : the x coordinate
@param b : the y coordinate
@param c : the z coordinate
@return a bitarray with 1s corresponding to cheese at index t
'''
def cheese_phases(a: int, b: int, c: int) -> bitarray:
    cheese_times = bitarray('0' * (n+1))
    prod_coords = a * b * c

    for x in range(n // 2):
        cheese_times[((A * (prod_coords + x) + C) % M) % n] = 1

    return cheese_times


start = time()
# set up the DP and maze
dp = np.full((k, k, k, n+1), float('-inf'))
maze = np.empty((k, k, k), dtype=object)

# pre compute all of the phases for each coordinate in the maze
for a, b, c in product(range(k), range(k), range(k)):
    maze[a][b][c] = cheese_phases(a + 1, b + 1, c + 1)

print(f"Setup took {round(time() - start, 3)} seconds\n")

'''
uses DP to calculate the path that collects the most amnount of cheese
@param a : the x coordinate
@param b : the y coordinate
@param c : the z coordinate
@param t : the time
@return the maximum amount of cheese that is collectible
'''
def dfs(a: int, b: int, c: int, t: int) -> int:
    if a >= k or b >= k or c >= k or t > n:
        return 0

    if dp[a][b][c][t] != float('-inf'):
        return dp[a][b][c][t]

    if maze[a][b][c][t]:
        max_cheese = 1 + max(
            dfs(a + 1, b, c, t + 1), #R
            dfs(a, b + 1, c, t + 1), #U
            dfs(a, b, c + 1, t + 1)  #F
        )

    else:
        max_cheese = max(
            dfs(a + 1, b, c, t + 1), #R
            dfs(a, b + 1, c, t + 1), #U
            dfs(a, b, c + 1, t + 1), #F
            dfs(a, b, c, t + 1)      #W
        )

    dp[a][b][c][t] = max_cheese
    return int(max_cheese)


start = time()
print(dfs(0, 0, 0, 1))
print(f"\nSolved in {round(time() - start, 3)} seconds")

Setup took 0.507 seconds

87

Solved in 8.987 seconds


In [ ]:
'''
searches over the dynamically stored optimal path to print out the path taken
@param a : the x coordinate
@param b : the y coordinate
@param c : the z coordinate
@param t : the time
@param total : the total cheese collected
@param path : the path taken by the mouse
'''
def path_search(a: int, b: int, c: int, t: int, total: int, path: str) -> None:
    if len(path) == n - 1:
        print(total)
        print(path)
    elif t <= n and a < k and b < k and c < k:
        if maze[a][b][c][t]:
            if a == k - 1 and b == k - 1 and c == k - 1:
                while len(path) < n - 1:
                    path += 'W'
                print(total + 1)
                print(path)
            elif t < n:
                possibilities = [
                    dp[a+1][b][c][t+1] if a+1<k else 0,
                    dp[a][b+1][c][t+1] if b+1<k else 0,
                    dp[a][b][c+1][t+1] if c+1<k else 0
                ]
                max_index = np.argmax(possibilities)

                if max_index==0: path_search(a + 1, b, c, t + 1, total + 1, path + 'R')
                elif max_index==1: path_search(a, b + 1, c, t + 1, total + 1, path + 'U')
                else: path_search(a, b, c + 1, t + 1, total + 1, path + 'F')

        elif t < n:
            possibilities = [
                    dp[a+1][b][c][t+1] if a+1<k else 0,
                    dp[a][b+1][c][t+1] if b+1<k else 0,
                    dp[a][b][c+1][t+1] if c+1<k else 0,
                    dp[a][b][c][t+1]
            ]
            max_index = np.argmax(possibilities)

            if max_index==0: path_search(a + 1, b, c, t + 1, total, path + 'R')
            elif max_index==1: path_search(a, b + 1, c, t + 1, total, path + 'U')
            elif max_index==2: path_search(a, b, c + 1, t + 1, total, path + 'F')
            else: path_search(a, b, c, t + 1, total, path + 'W')


start = time()
path_search(0, 0, 0, 1, 0, '')
print(f"\nSolved in {round(time() - start, 3)} seconds")

87
RRWRWWUUWUFFRWRRRFRWRRRRRUUWRRRRRRFUURUURUUUUUUUUFRRRRRFUUFFFFFFFFFFUUUFFFRUURUFFFUFUFRUUFFFFWWFWWW

Solved in 0.002 seconds


In [ ]:
# check to make sure length is correct
print(len("RRWRWWUUWUFFRWRRRFRWRRRRRUUWRRRRRRFUURUURUUUUUUUUFRRRRRFUUFFFFFFFFFFUUUFFFRUURUFFFUFUFRUUFFFFWWFWWW"))

99


In [ ]:
'''
Double checks to make sure the path returned gives the max value found
@param path : the path that the mouse took
@return the amount of cheese collected on the given path
'''
def recheck(path: str) -> int:
    seen = set()
    a,b,c = 0,0,0
    t,tot = 1,0
    if maze[a][b][c][t] and (a,b,c) not in seen:
        tot += 1
        seen.add((a,b,c))
    for ch in path:
        if ch=='R': a+=1
        elif ch=='U': b+=1
        elif ch=='F': c+=1
        t += 1
        if maze[a][b][c][t] and (a,b,c) not in seen:
            tot += 1
            seen.add((a,b,c))
    return tot


recheck("RRWRWWUUWUFFRWRRRFRWRRRRRUUWRRRRRRFUURUURUUUUUUUUFRRRRRFUUFFFFFFFFFFUUUFFFRUURUFFFUFUFRUUFFFFWWFWWW")

87

In [ ]:
# now we need to change some variables for the bonus version
k = 50
n = 200

In [ ]:
'''
calculates the times when cheese is available at a given coordinate
@param a : the x coordinate
@param b : the y coordinate
@param c : the z coordinate
@return a bitarray with 1s corresponding to cheese at index t
'''
def cheese_phases(a: int, b: int, c: int) -> bitarray:
    cheese_times = bitarray('0' * (n+1))
    prod_coords = a * b * c

    for x in range(n // 2):
        cheese_times[((A * (prod_coords + x) + C) % M) % n] = 1

    return cheese_times


start = time()
# set up the DP and maze
dp = np.full((k, k, k, n+1), float('-inf'))
maze = np.empty((k, k, k), dtype=object)


# pre compute all of the phases for each coordinate in the maze
for a, b, c in product(range(k), range(k), range(k)):
    maze[a][b][c] = cheese_phases(a + 1, b + 1, c + 1)

print(f"Setup took {round(time() - start, 3)} seconds\n")


'''
uses DP to calculate the path that collects the most amnount of cheese
@param a : the x coordinate
@param b : the y coordinate
@param c : the z coordinate
@param t : the time
@return the maximum amount of cheese that is collectible
'''
def dfs_bonus(a: int, b: int, c: int, t: int) -> int:
    if a >= k or b >= k or c >= k or t > n or a < 0 or b < 0 or c < 0:
        return 0

    if dp[a][b][c][t] != float('-inf'):
        return dp[a][b][c][t]

    max_cheese = maze[a][b][c][t] + max(
                dfs(a + 1, b, c, t + 1), #R
                dfs(a - 1, b, c, t + 1), #L
                dfs(a, b + 1, c, t + 1), #U
                dfs(a, b - 1, c, t + 1), #D
                dfs(a, b, c + 1, t + 1), #F
                dfs(a, b, c - 1, t + 1), #B
                dfs(a, b, c, t + 1)      #W
    )

    dp[a][b][c][t] = max_cheese
    return int(max_cheese)


start = time()
print(dfs_bonus(0, 0, 0, 1))
print(f"\nSolved in {round(time() - start, 3)} seconds")

Setup took 4.423 seconds

184

Solved in 216.173 seconds


In [ ]:
'''
searches over the dynamically stored optimal path to print out the path taken
@param a : the x coordinate
@param b : the y coordinate
@param c : the z coordinate
@param t : the time
@param total : the total cheese collected
@param path : the path taken by the mouse
'''
def path_search(a: int, b: int, c: int, t: int, total: int, path: str) -> None:
    if len(path) == n - 1:
        print(total)
        print(path)

    elif t < n and a < k and b < k and c < k and a >= 0 and b >= 0 and c >= 0:

        possibilities = [
                dp[a+1][b][c][t+1] if a+1<k else 0,
                dp[a-1][b][c][t+1] if a>0 else 0,
                dp[a][b+1][c][t+1] if b+1<k else 0,
                dp[a][b-1][c][t+1] if b>0 else 0,
                dp[a][b][c+1][t+1] if c+1<k else 0,
                dp[a][b][c-1][t+1] if c>0 else 0,
                dp[a][b][c][t+1]
        ]

        max_index = np.argmax(possibilities)

        if max_index==0: path_search(a + 1, b, c, t + 1, total + maze[a][b][c][t], path + 'R')
        elif max_index==1: path_search(a - 1, b, c, t + 1, total + maze[a][b][c][t], path + 'L')
        elif max_index==2: path_search(a, b + 1, c, t + 1, total + maze[a][b][c][t], path + 'U')
        elif max_index==3: path_search(a, b - 1, c, t + 1, total + maze[a][b][c][t], path + 'D')
        elif max_index==4: path_search(a, b, c + 1, t + 1, total + maze[a][b][c][t], path + 'F')
        elif max_index==5: path_search(a, b, c - 1, t + 1, total + maze[a][b][c][t], path + 'B')
        else: path_search(a, b, c, t + 1, total + maze[a][b][c][t], path + 'W')


start = time()
path_search(0, 0, 0, 1, 0, '')
print(f"\nSolved in {round(time() - start, 3)} seconds")

184
RRRRRRRRRRRRURRRRUUFRRRUUUURFBUUBFRFLFLFLFWFBBFWFWUWFDBUDBUWFBBFBBFWFBBFBBFLFWFWFLFWFWULBUWULBFBBFWRWFWFBBFLFDBFDBRWFBBFWFWUWUDDUBBFWUWFWUBBBFLFBBFBBFWFBBFWFWRLBFLFWFWRWFBBFLBRWFBBFLBRLBFDFBBFBBFWFLR

Solved in 0.004 seconds


In [ ]:
# check to make sure length is correct
len("RRRRRRRRRRRRURRRRUUFRRRUUUURFBUUBFRFLFLFLFWFBBFWFWUWFDBUDBUWFBBFBBFWFBBFBBFLFWFWFLFWFWULBUWULBFBBFWRWFWFBBFLFDBFDBRWFBBFWFWUWUDDUBBFWUWFWUBBBFLFBBFBBFWFBBFWFWRLBFLFWFWRWFBBFLBRWFBBFLBRLBFDFBBFBBFWFLR")

199

In [ ]:
'''
Double checks to make sure the path returned gives the max value found
@param path : the path that the mouse took
@return the amount of cheese collected on the given path
'''
def recheck_bonus(path: str) -> int:
  a,b,c = 0,0,0
  t,tot = 1,0
  if maze[a][b][c][t]:
      tot += 1
  for ch in path:
      if ch=='R':a+=1
      elif ch=='L':a-=1
      elif ch=='U':b+=1
      elif ch=='D':b-=1
      elif ch=='F':c+=1
      elif ch=='B':c-=1
      t += 1
      tot += maze[a][b][c][t]
  return tot


recheck_bonus("RRRRRRRRRRRRURRRRUUFRRRUUUURFBUUBFRFLFLFLFWFBBFWFWUWFDBUDBUWFBBFBBFWFBBFBBFLFWFWFLFWFWULBUWULBFBBFWRWFWFBBFLFDBFDBRWFBBFWFWUWUDDUBBFWUWFWUBBBFLFBBFBBFWFBBFWFWRLBFLFWFWRWFBBFLBRWFBBFLBRLBFDFBBFBBFWFLR")

184